# **DPO Evaluation**

## **Config**

In [1]:
import os
seed = 42
gpu_ids = "7"

CACHE_FOLDER    = "../model_cache"
METRICS_FILE    = "dpo_evaluation_metrics.csv"
DATASET_PATH    = "../datasets_full_training_and_test/cqa_test_prompt_completion.jsonl"
BATCH_SIZE      = 128

# model_key → final merged DPO checkpoint dir
MODEL_NAMES = {
#     "Meta-Llama-3.1-8B": os.path.join("dpo_output_Meta-Llama-3.1-8B", "final_checkpoint"),
#     "Aya-23-8B":     os.path.join("dpo_output_Aya-23-8B", "final_checkpoint"),
#     "SeaLLMs-v3-7B": os.path.join("dpo_output_SeaLLMs-v3-7B", "final_checkpoint"),
#     "SEA-LION-v3-8B":    os.path.join("dpo_output_SEA-LION-v3-8B", "final_checkpoint"),
    "Sahabat-AI-8B":     os.path.join("dpo_output_Sahabat-AI-8B", "final_checkpoint"),
}

## **Import Libraries**

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_ids

import gc, time, random
import torch, numpy as np, pandas as pd
from tqdm.auto import tqdm
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    set_seed
)
import evaluate

pd.set_option("display.max_colwidth", None)

/raid/home/llmsosmed/rlaif/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-06-09 07:01:29,094] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


## **Utility Functions**

In [3]:
def set_global_seed(s: int = seed):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark     = False
    set_seed(s)

def load_dpo_model_and_tokenizer(model_path: str, tokenizer_path: str, hf_token: str = None):
    # reset seed
    set_global_seed()
    # tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        tokenizer_path,
        cache_dir=CACHE_FOLDER,
        use_auth_token=hf_token,
        local_files_only=True
    )
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"
    # model (4-bit)
    quant_cfg = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        quantization_config=quant_cfg,
        low_cpu_mem_usage=True,
        use_auth_token=hf_token,
        local_files_only=True
    )
    model.config.use_cache = False
    model.eval()
    return tokenizer, model

def generate_dpo_batch(tokenizer, model, prompts, max_new_tokens=256):
    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=1024
    ).to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    # strip off the prompt and take only first line
    return [
        text[len(prompt):]#.split("\n", 1)[0]
        for text, prompt in zip(decoded, prompts)
    ]

set_global_seed()

## **Load Dataset**

In [4]:
df = pd.read_json(DATASET_PATH, lines=True)
print(f"Total examples: {len(df)}")
df.sample(3)

Total examples: 340


,prompt,completion
284,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nPasal 15A Yang dimaksud dengan “Reksa Dana berbasis Efek luar negeri” adalah Reksa Dana berbasis Efek luar negeri dalam bentuk konvensional dan syariah.\n\nQuestion: Apa pengertian Reksa Dana berbasis Efek luar negeri menurut Pasal 15A?\nAnswer:",Reksa Dana berbasis Efek luar negeri adalah Reksa Dana berbasis Efek luar negeri dalam bentuk konvensional dan syariah.
116,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nPasal 5\n(1)\nSitus Web wajib memuat informasi mengenai Emiten\natau Perusahaan Publik yang terbuka untuk umum,\naktual, dan terkini.\n(2)\nInformasi sebagaimana dimaksud pada ayat (1) wajib:\na. disajikan dengan benar dan tidak menyesatkan\nmengenai keadaan Emiten atau Perusahaan Publik;\nb. disajikan secara jelas sehingga mudah dipahami;\ndan\nc. dapat diakses setiap saat oleh semua pihak.\nPasal 6\nInformasi yang wajib dimuat dalam Situs Web Emiten atau\nPerusahaan Publik sebagaimana dimaksud dalam Pasal 5\npaling kurang meliputi:\na. informasi umum Emiten atau Perusahaan Publik;\nb. informasi bagi pemodal atau investor;\nc. informasi tata kelola perusahaan; dan\nd. informasi tanggung jawab sosial perusahaan.\n\nQuestion: Apa saja kategori penilaian informasi yang wajib dimuat dalam situs web emiten?\nAnswer:","Kategori penilaian meliputi informasi umum emiten atau perusahaan publik, informasi bagi pemodal atau investor, informasi tata kelola perusahaan, dan informasi tanggung jawab sosial perusahaan."
113,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nPasal 1 Dalam Peraturan Otoritas Jasa Keuangan ini, yang dimaksud dengan: 1. Dana Pensiun Pemberi Kerja adalah dana pensiun yang dibentuk oleh orang atau badan yang mempekerjakan karyawan, selaku pendiri, untuk menyelenggarakan program pensiun manfaat pasti atau program pensiun iuran pasti, bagi kepentingan sebagian atau seluruh karyawannya sebagai peserta, dan yang menimbulkan kewajiban terhadap pemberi kerja. 2. Peraturan Dana Pensiun adalah peraturan yang berisi ketentuan yang menjadi dasar penyelenggaraan program pensiun. 3. Otoritas Jasa Keuangan yang selanjutnya disingkat OJK adalah Otoritas Jasa Keuangan sebagaimana dimaksud dalam Undang-Undang Nomor 21 Tahun 2011 tentang Otoritas Jasa Keuangan.\n\nQuestion: Apa tujuan utama OJK dalam Pasal 1?\nAnswer:",Saya tidak tahu terkait tujuan utama OJK dalam Pasal 1.


## **Prepare Metrics**

In [5]:
if not os.path.exists(METRICS_FILE):
    pd.DataFrame(columns=[
        "model","exact_match","rouge1_f1","rouge2_f1","rougeL_f1",
        "bleu","meteor","inference_time_sec"
    ]).to_csv(METRICS_FILE, index=False)

# evaluators
em    = evaluate.load("exact_match")
rouge = evaluate.load("rouge")
bleu  = evaluate.load("bleu")
meteor= evaluate.load("meteor")

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/llmsosmed/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/llmsosmed/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/llmsosmed/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## **Inference & Evaluation**

In [6]:
for model_key, model_id in MODEL_NAMES.items():
    print(f"\n▶ Evaluating DPO {model_key}")
    tokenizer_dir = os.path.join(CACHE_FOLDER, model_key)
    tokenizer, model = load_dpo_model_and_tokenizer(
        model_id, tokenizer_dir, hf_token=os.getenv("HF_TOKEN")
    )

    # LoRA weights (sanity check)
    print("=== LoRA weights for", model_key, "===")
    for name, param in model.named_parameters():
        if "lora" in name:
            print(name, param.norm().item())
    print("="*30)

    # Warm-up
    _ = model.generate(
        **tokenizer("Warm up", return_tensors="pt").to(model.device),
        max_new_tokens=1,
        pad_token_id=tokenizer.eos_token_id
    )

    preds, refs, details = [], [], []
    start = time.time()

    for i in tqdm(range(0, len(df), BATCH_SIZE), desc=model_key):
        batch    = df.iloc[i : i + BATCH_SIZE]
        prompts  = batch["prompt"].tolist()
        batch_preds = generate_dpo_batch(tokenizer, model, prompts)

        preds.extend(batch_preds)
        refs.extend(batch["completion"].tolist())

        for prompt, ref, pred in zip(prompts, batch["completion"], batch_preds):
            details.append({
                "prompt":       prompt,
                "ground_truth": ref,
                model_key:      pred
            })

    inf_time = time.time() - start
    print(f"→ Inference time: {inf_time:.1f} sec")

    # compute
    r_em  = em.compute(predictions=preds, references=refs)["exact_match"]
    r_rog = rouge.compute(predictions=preds, references=refs)
    r_ble = bleu.compute(predictions=preds, references=[[r] for r in refs])["bleu"]
    r_met = meteor.compute(predictions=preds, references=refs)["meteor"]

    # append
    row = {
        "model":             model_key,
        "exact_match":       r_em,
        "rouge1_f1":         r_rog["rouge1"],
        "rouge2_f1":         r_rog["rouge2"],
        "rougeL_f1":         r_rog["rougeL"],
        "bleu":              r_ble,
        "meteor":            r_met,
        "inference_time_sec": inf_time
    }
    pd.DataFrame([row]).to_csv(METRICS_FILE, mode="a", header=False, index=False)

    # detailed
    pd.DataFrame(details).to_json(
        f"dpo_detailed_{model_key}.jsonl", 
        orient="records", lines=True
    )

    # cleanup
    del model, tokenizer
    torch.cuda.empty_cache()
    gc.collect()

## **Results**

In [7]:
dfm = pd.read_csv(METRICS_FILE)
print(dfm.to_markdown(index=False))

| model             |   exact_match |   rouge1_f1 |   rouge2_f1 |   rougeL_f1 |      bleu |   meteor |   inference_time_sec |
|:------------------|--------------:|------------:|------------:|------------:|----------:|---------:|---------------------:|
| Meta-Llama-3.1-8B |      0        |    0.211001 |    0.165594 |    0.198789 | 0.0820524 | 0.434379 |              251.354 |
| Aya-23-8B         |      0        |    0.167482 |    0.129123 |    0.159358 | 0.0605214 | 0.367553 |              213.159 |
| SeaLLMs-v3-7B     |      0.267647 |    0.613463 |    0.539295 |    0.595446 | 0.244062  | 0.675176 |              192.205 |
| SEA-LION-v3-8B    |      0        |    0.21992  |    0.171337 |    0.207273 | 0.0850537 | 0.439728 |              251.314 |
| Sahabat-AI-8B     |      0.220588 |    0.532757 |    0.468963 |    0.516878 | 0.185292  | 0.617425 |             7991.42  |


In [8]:
merged = None

for key in MODEL_NAMES:
    # base
    base_path = f"../base-evaluation/detailed_{key}.jsonl"
    if os.path.exists(base_path):
        base_df = (
            pd.read_json(base_path, lines=True)
              .rename(columns={key: f"pred_base_{key}"})
              [["prompt","ground_truth",f"pred_base_{key}"]]
        )
        merged = base_df if merged is None else merged.merge(
            base_df, on=["prompt","ground_truth"], how="outer"
        )

    # sft
    sft_path = f"../sft/sft_detailed_{key}.jsonl"
    if os.path.exists(sft_path):
        sft_df = (
            pd.read_json(sft_path, lines=True)
              .rename(columns={key: f"pred_sft_{key}"})
              [["prompt","ground_truth",f"pred_sft_{key}"]]
        )
        merged = sft_df if merged is None else merged.merge(
            sft_df, on=["prompt","ground_truth"], how="outer"
        )

    # dpo
    dpo_path = f"dpo_detailed_{key}.jsonl"
    if os.path.exists(dpo_path):
        dpo_df = (
            pd.read_json(dpo_path, lines=True)
              .rename(columns={key: f"pred_dpo_{key}"})
              [["prompt","ground_truth",f"pred_dpo_{key}"]]
        )
        merged = dpo_df if merged is None else merged.merge(
            dpo_df, on=["prompt","ground_truth"], how="outer"
        )

print(f"Total rows in merged DataFrame: {len(merged)}")
display(merged.head(5))

Total rows in merged DataFrame: 340


,prompt,ground_truth,pred_base_Sahabat-AI-8B,pred_sft_Sahabat-AI-8B,pred_dpo_Sahabat-AI-8B
0,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\n(1) BPR dan BPRS wajib menyusun Rencana Bisnis secara realistis setiap tahun. (2) Rencana Bisnis sebagaimana dimaksud pada ayat (1) wajib disusun oleh Direksi dan disetujui oleh Dewan Komisaris.\n\n(3) Rencana Bisnis yang disusun oleh BPR dan BPRS sebagaimana dimaksud pada ayat (1) harus mencerminkan arah dan kebijakan pengembangan usaha BPR dan BPRS dalam jangka pendek, jangka menengah, dan jangka panjang. (4) Arah dan kebijakan pengembangan usaha BPR dan BPRS dalam jangka panjang sebagaimana dimaksud pada ayat (3) meliputi visi dan misi BPR dan BPRS.\n\nQuestion: Bagaimana mekanisme pelaporan audit internal pada Pasal 2?\nAnswer:",Saya tidak tahu terkait mekanisme pelaporan audit internal pada Pasal 2.,Saya tidak tahu terkait mekanisme pelaporan audit internal pada Pasal 2. Pasal tersebut tidak menyebutkan secara spesifik tentang mekanisme pelaporan audit internal.,Saya tidak tahu terkait mekanisme pelaporan audit internal pada Pasal 2.,Saya tidak tahu terkait mekanisme pelaporan audit internal pada Pasal 2.
1,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\n(1) BPR dan BPRS wajib menyusun Rencana Bisnis secara realistis setiap tahun. (2) Rencana Bisnis sebagaimana dimaksud pada ayat (1) wajib disusun oleh Direksi dan disetujui oleh Dewan Komisaris.\n\n(3) Rencana Bisnis yang disusun oleh BPR dan BPRS sebagaimana dimaksud pada ayat (1) harus mencerminkan arah dan kebijakan pengembangan usaha BPR dan BPRS dalam jangka pendek, jangka menengah, dan jangka panjang. (4) Arah dan kebijakan pengembangan usaha BPR dan BPRS dalam jangka panjang sebagaimana dimaksud pada ayat (3) meliputi visi dan misi BPR dan BPRS.\n\nQuestion: Siapa yang wajib menyusun Rencana Bisnis menurut Pasal 2 ayat (2)?\nAnswer:",Mekanisme proses dijelaskan sebagai Direksi wajib menyusun Rencana Bisnis dan disetujui oleh Dewan Komisaris.,"Direksi dan disetujui oleh Dewan Komisaris.\n\nQuestion: Apa yang harus mencerminkan arah dan kebijakan pengembangan usaha BPR dan BPRS dalam jangka panjang?\nAnswer: Visi dan misi BPR dan BPRS.\n\nQuestion: Apakah BPR dan BPRS wajib menyusun Rencana Bisnis secara realistis setiap tahun?\nAnswer: Ya.\n\nQuestion: Apakah Rencana Bisnis yang disusun oleh BPR dan BPRS harus mencerminkan arah dan kebijakan pengembangan usaha BPR dan BPRS dalam jangka pendek, jangka menengah, dan jangka panjang?\nAnswer: Ya.\n\nQuestion: Apakah arah dan kebijakan pengembangan usaha BPR dan BPRS dalam jangka panjang meliputi visi dan misi BPR dan BPRS?\nAnswer: Ya.\n\nQuestion: Apakah Direksi dan Dewan Komisaris harus menyusun Rencana Bisnis yang mencerminkan arah dan kebijakan pengembangan usaha BPR dan BPRS dalam jangka panjang?\nAnswer: Ya.\n\nQuestion:","Direksi dan disetujui oleh Dewan Komisaris.\n\nQuestion: Apa yang harus mencerminkan arah dan kebijakan pengembangan usaha BPR dan BPRS dalam jangka panjang?\nAnswer: Visi dan misi BPR dan BPRS.\n\nQuestion: Apakah BPR dan BPRS wajib menyusun Rencana Bisnis secara realistis setiap tahun?\nAnswer: Ya.\n\nQuestion: Apakah Rencana Bisnis harus disusun oleh Direksi dan disetujui oleh Dewan Komisaris?\nAnswer: Ya.\n\nQuestion: Apakah Rencana Bisnis harus mencerminkan arah dan kebijakan pengembangan usaha BPR dan BPRS dalam jangka pendek, jangka menengah, dan jangka panjang?\nAnswer: Ya.\n\nQuestion: Apakah arah dan kebijakan pengembangan usaha BPR dan BPRS dalam jangka panjang meliputi visi dan misi BPR dan BPRS?\nAnswer: Ya.\n\nQuestion: Apakah BPR dan BPRS wajib menyusun Rencana Bisnis secara realistis setiap tahun?\nAnswer: Ya.\n\nQuestion","Direksi dan disetujui oleh Dewan Komisaris.\n\nQu

In [ ]:
import signal

os.kill(os.getpid(), signal.SIGTERM)